In [7]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
import lightgbm
import catboost
import xgboost
import shap
#import seaborn as sns
from catboost import *
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import r2_score
#%matplotlib
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=np.sqrt(sum(squaredError)/len(squaredError))
    R2=r2_score(target,prediction)
    return mae,mse,RMSE,R2
###########loading data##########
fdata=pd.read_csv('database_filled_ST.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                      'Operating Temperature (℃)',#0
                      'Flow Rate (mL min-1)',#1
                      'Active Area (cm2)', #2
                      'Ir wt. %',#3
                      'Ru wt.%',#4
                      'O wt. %',#5
                      'I/C in Anode',#6
                      'Pt wt. %',#7
                      'I/C in Cathode',#8 
                      'Anode Precious Metal Loading (mg cm-2 Ir/Ru/Pt/Pd)',#9
                      'Cathode Precious Metal Loading (mg cm-2 Pt/Pd)',#10
                      'Membrane Thickness (μm)',#11
                      'EW',#12
                      'Maximum Stability Current Density (A cm-2)',#13
                      'Stability Test Time (h-1)'#14
                        ]]
###########train test splitting##########
raw_param=raw_data.iloc[:,0:15]
print('ready')


ready


In [13]:
DR=fdata.loc[:,['logDR']]

In [14]:
def summarize(list1):
    total=0
    for ele in range(0, len(list1)):
        total = total + list1[ele]
    return total

In [15]:
def gridsearch(model,param,algorithm_name):
    grid = GridSearchCV(model,param_grid=param,cv=5,n_jobs=-1)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    result = best_model.predict(X_test)
    x_prediction_07=result
    y_real_07=y_test[:,0]
    x_prediction_07_series=pd.Series(x_prediction_07)
    y_real_07_series=pd.Series(y_real_07)
    
    result_train = best_model.predict(X_train)
    x_prediction_07_train=result_train
    y_real_07_train=y_train[:,0]
    x_prediction_07_series_train=pd.Series(x_prediction_07_train)
    y_real_07_series_train=pd.Series(y_real_07_train)
    
    ###########evaluating the regression quality##########
    corr_ann = round(x_prediction_07_series.corr(y_real_07_series), 5)
    error_val= compute_mae_mse_rmse(x_prediction_07,y_real_07)
    
    corr_ann_train = round(x_prediction_07_series_train.corr(y_real_07_series_train), 5)
    error_val_train= compute_mae_mse_rmse(x_prediction_07_train,y_real_07_train)
    
    print(algorithm_name)
    print(best_model.feature_importances_)
    print('Best Regressor:',grid.best_params_,'Best Score:', grid.best_score_)
    print(error_val,'TEST R2',error_val[3],'TEST CORR',corr_ann)
    print(error_val_train,'TRAIN R2',error_val_train[3],'TRAIN CORR',corr_ann_train)
    x_y_x=np.arange(0,5,0.1)
    x_y_y=np.arange(0,5,0.1)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(x_prediction_07,y_real_07,color='red',label=algorithm_name+' Test Set',alpha=0.75)
    ax.scatter(x_prediction_07_train,y_real_07_train,color='blue',label=algorithm_name+' Training Set',alpha=0.25,marker="^")
    ax.plot(x_y_x,x_y_y)
    plt.legend()
    plt.xlabel(u"Predicted_Log(Decay Rate(μV h-1))")
    plt.ylabel(u"Real_Log(Decay Rate(μV h-1))")
    plt.savefig('DR SEL %s.png' %algorithm_name)
    print('finished')

In [16]:
seed= 1686
X_train, X_test, y_train, y_test = train_test_split(raw_param, DR, test_size=.15,random_state=seed)

In [18]:
train=pd.concat([X_train,y_train],axis=1)
train.to_csv('DCREG_train.csv')
test=pd.concat([X_test,y_test],axis=1)
test.to_csv('DCREG_test.csv')

In [21]:
from catboost import *
import shap
def shap_plot(model,param,algorithm_name):
    print(algorithm_name)
    SHAP_INPUT=raw_data.iloc[:,0:15]
    SHAP_OUTPUT=DR
    grid = GridSearchCV(model,param_grid=param,cv=5,n_jobs=-100)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT
    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        shap_values=shap_values[:,:-1]
        shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values).mean(0)
        print(global_importances)
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values).mean(0)
        print(global_importances)
    else:
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values).mean(0)
        print(global_importances)

In [22]:
##########LGBM gridsearch CV for best hyperparameter##########
model_LightGBMRegressor=lightgbm.LGBMRegressor(random_state=1,verbose=-1)
param_light = {
'boosting_type':['gbdt','rf'],
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
'subsample':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
'n_estimators':[50,100,200,400],
'max_depth':[5,7,9,11,13,-1],
'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
}
gridsearch(model_LightGBMRegressor,param_light,'LightGBM')

##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBRegressor=xgboost.XGBRegressor(objective='reg:squarederror',random_state=1,verbosity=0)
param_xg = {
'booster':['gbtree'],
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
'n_estimators':[100,200,400],
'max_depth':[5,7,9,11,13],
'subsample':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
}
gridsearch(model_XGBRegressor,param_xg,'XGBoost')

##########CatBoost gridsearch CV for best hyperparameter##########
model_CatRegressor=catboost.CatBoostRegressor(random_state=1,verbose=0)
param_cat = {
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5],
'n_estimators':[50,100,200,400],
"boosting_type":["Plain"],
'max_depth':[5,7,9,11],
'subsample':[0.4,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
}
gridsearch(model_CatRegressor,param_cat,'CatBoost')


###########GradientBoost gridsearch CV for best hyperparameter##########
model_GradientBoostingRegressor = ensemble.GradientBoostingRegressor(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
'n_estimators':[50,100,200,400],
'max_depth':[3,5,7,9,11,13,16],
'criterion':['friedman_mse','mae','mse'],
'max_features':['auto','sqrt','log2'],
'loss':['ls', 'lad', 'huber', 'quantile']
}
gridsearch(model_GradientBoostingRegressor,param_GB,'GradientBoost')

###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestRegressor = ensemble.RandomForestRegressor(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
'n_estimators':[50,100,200,400,None],
'max_depth':[3,5,7,9,11,None],
'criterion':['mse','mae'],
'max_features':['auto','sqrt','log2']
}
gridsearch(model_RandomForestRegressor,param_RF,'Random Forest')


###########Extra Tree gridsearch CV for best hyperparameter##########
model_ExtraTreeRegressor = ExtraTreeRegressor(random_state=1)
param_ET = {
'max_depth':[5,6,7,8,9,10,11,None],
'max_features':['auto','sqrt','log2'],
'criterion' : ["mse", "friedman_mse", "mae"],
'splitter' : [ "best",'random']
}
gridsearch(model_ExtraTreeRegressor,param_ET,'Extra Tree')


###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeRegressor = tree.DecisionTreeRegressor(random_state=1)
param_DT = {
'max_depth':[5,6,7,8,9,10,11,None],
'max_features':['auto','sqrt','log2'],
'criterion' : ["mse", "friedman_mse", "mae"],
'splitter' : [ "best",'random']
}
gridsearch(model_DecisionTreeRegressor,param_DT,'Decision Tree')


###########AdaBoost gridsearch CV for best hyperparameter##########
model_AdaBoostRegressor = ensemble.AdaBoostRegressor(random_state=1)
param_Ada = {
'n_estimators':[50,100,200,400,800],
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
'loss':['linear', 'square', 'exponential']
}
gridsearch(model_AdaBoostRegressor,param_Ada,'AdaBoost')

In [23]:
##########LGBM gridsearch CV for best hyperparameter##########
model_LightGBMRegressor=lightgbm.LGBMRegressor(random_state=1,verbose=-1)
param_light = {
'boosting_type':['gbdt'],
'learning_rate':[0.06],
'subsample':[0.4],
'n_estimators':[200],
'max_depth':[5],
'reg_alpha':[1e-05],
'reg_lambda':[0.0001]
}
gridsearch(model_LightGBMRegressor,param_light,'LightGBM')

Converting column-vector to 1d array


LightGBM
[ 0 24 50  2 38 32 50 17 34 27 22 25  0  1 56]
Best Regressor: {'boosting_type': 'gbdt', 'learning_rate': 0.06, 'max_depth': 5, 'n_estimators': 200, 'reg_alpha': 1e-05, 'reg_lambda': 0.0001, 'subsample': 0.4} Best Score: 0.30372529341095794
(0.2665109717947528, 0.11504808886093473, 0.33918739490278044, 0.8644766942152746) TEST R2 0.8644766942152746 TEST CORR 0.94501
(0.40455805059012373, 0.2489764414797677, 0.4989753916575122, 0.6381105320039602) TRAIN R2 0.6381105320039602 TRAIN CORR 0.88155
finished


In [24]:
%matplotlib
shap_plot(model_LightGBMRegressor,param_light,'LightGBM')

Using matplotlib backend: Qt5Agg
LightGBM


Converting column-vector to 1d array
Converting column-vector to 1d array
Converting column-vector to 1d array
Converting column-vector to 1d array
Converting column-vector to 1d array
Converting column-vector to 1d array


[0.         0.06461962 0.37839104 0.00440443 0.15478283 0.29711558
 0.13686319 0.07221193 0.17395768 0.03187179 0.16839619 0.15952983
 0.         0.00934525 0.21033057]


In [25]:
##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBRegressor=xgboost.XGBRegressor(objective='reg:squarederror',random_state=1,verbosity=0)
param_xg = {
'booster':['gbtree'],
'learning_rate':[0.15],
'n_estimators':[200],
'max_depth':[13],
'subsample':[0.55],
'reg_alpha':[0.0001],
'reg_lambda':[0.001]
}
gridsearch(model_XGBRegressor,param_xg,'XGBoost')

XGBoost
[0.01191314 0.02487898 0.19678505 0.05210716 0.01168877 0.17417645
 0.07979202 0.02799346 0.06338507 0.06297662 0.06090068 0.04531766
 0.03006596 0.03617569 0.12184319]
Best Regressor: {'booster': 'gbtree', 'learning_rate': 0.15, 'max_depth': 13, 'n_estimators': 200, 'reg_alpha': 0.0001, 'reg_lambda': 0.001, 'subsample': 0.55} Best Score: 0.48777467502927196
(0.22115186209064813, 0.08199583484538508, 0.28634914849774756, 0.9145568321974602) TEST R2 0.9145568321974602 TEST CORR 0.96227
(0.012850000644762449, 0.0031365805057806556, 0.056005182847488816, 0.9970801954640574) TRAIN R2 0.9970801954640574 TRAIN CORR 0.99856
finished


In [26]:
shap_plot(model_XGBRegressor,param_xg,'XGBoost')

XGBoost
[0.03558253 0.10603233 0.2861385  0.06218899 0.00687563 0.22133228
 0.08578311 0.037913   0.07620198 0.09431049 0.04002416 0.02095892
 0.00461397 0.0638824  0.2275378 ]


In [29]:
##########CatBoost gridsearch CV for best hyperparameter##########
model_CatRegressor=catboost.CatBoostRegressor(random_state=1,verbose=0)
param_cat = {

'max_depth':[11],
'boosting_type':['Plain'],
'subsample':[0.65],
'reg_lambda':[0]
}
gridsearch(model_CatRegressor,param_cat,'CatBoost')

CatBoost
[ 1.86814782  6.57098859 16.34942717  5.2648752   3.94689594  8.36574924
  4.32547603  6.24673805  6.96815082  6.39191042  4.40646197  6.39999174
  2.98957699  8.59791428 11.30769575]
Best Regressor: {'boosting_type': 'Plain', 'max_depth': 11, 'reg_lambda': 0, 'subsample': 0.65} Best Score: -20.15849450304906
(0.17760343945457674, 0.06742660019327697, 0.25966632471939244, 0.9136662857216502) TEST R2 0.9136662857216502 TEST CORR 0.96816
(0.008917702853200182, 0.0029025649403131956, 0.05387545768077702, 0.9973115583660213) TRAIN R2 0.9973115583660213 TRAIN CORR 0.99866
finished


In [30]:
shap_plot(model_CatRegressor,param_cat,'CatBoost')

CatBoost
[0.02635268 0.05655991 0.15253439 0.04218951 0.03829247 0.11940487
 0.04850698 0.07364819 0.05834663 0.05985974 0.06545051 0.06089543
 0.05500334 0.0697314  0.09836001]


In [31]:
###########GradientBoost gridsearch CV for best hyperparameter##########
model_GradientBoostingRegressor = ensemble.GradientBoostingRegressor(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
'learning_rate':[0.1],
'n_estimators':[400],
'max_depth':[11],
'criterion':['mse'],
'max_features':['sqrt'],
'loss':['quantile']
}
gridsearch(model_GradientBoostingRegressor,param_GB,'GradientBoost')

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


GradientBoost
[0.05564935 0.07307428 0.04314727 0.04537011 0.02494465 0.05633515
 0.09440341 0.03240859 0.07876219 0.09559086 0.07163013 0.04520581
 0.02825102 0.07922444 0.17600276]
Best Regressor: {'criterion': 'mse', 'learning_rate': 0.1, 'loss': 'quantile', 'max_depth': 11, 'max_features': 'sqrt', 'n_estimators': 400} Best Score: 0.334459555176188
(0.2792443309713643, 0.1288194677933337, 0.35891429031641203, 0.8467320962941498) TEST R2 0.8467320962941498 TEST CORR 0.94663
(0.11231889053109292, 0.07407082864928734, 0.2721595646845566, 0.9184602566573219) TRAIN R2 0.9184602566573219 TRAIN CORR 0.97286
finished


In [32]:
shap_plot(model_GradientBoostingRegressor,param_GB,'GradientBoost')

GradientBoost


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


[0.02210176 0.06935578 0.13547114 0.05516303 0.02336675 0.17363564
 0.07083967 0.07668154 0.05049795 0.05188605 0.05697716 0.05451707
 0.03261274 0.05282403 0.11296967]


In [28]:
###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestRegressor = ensemble.RandomForestRegressor(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
'n_estimators':[50],
'max_depth':[9],
'criterion':['mse'],
'max_features':['sqrt']
}
gridsearch(model_RandomForestRegressor,param_RF,'Random Forest')

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Random Forest
[0.02510884 0.08175031 0.14801534 0.03682191 0.03464239 0.17432893
 0.0607983  0.05624323 0.05986745 0.05990028 0.06508367 0.03178176
 0.01684793 0.04884574 0.09996392]
Best Regressor: {'criterion': 'mse', 'max_depth': 9, 'max_features': 'sqrt', 'n_estimators': 50} Best Score: 0.4340128075672885
(0.22521945380074823, 0.07342660703486904, 0.27097344341257695, 0.9006537187538614) TEST R2 0.9006537187538614 TEST CORR 0.96141
(0.24640015128587378, 0.10075760203694814, 0.3174233797894354, 0.8546967304892686) TRAIN R2 0.8546967304892686 TRAIN CORR 0.96692
finished


In [33]:
shap_plot(model_RandomForestRegressor,param_RF,'Random Forest')

Random Forest


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


[0.02235735 0.08924829 0.16248124 0.0304823  0.03964319 0.15657047
 0.06268738 0.044415   0.05376569 0.03809887 0.04256409 0.04218857
 0.04133903 0.02974314 0.0852461 ]


In [34]:
###########Extra Tree gridsearch CV for best hyperparameter##########
model_ExtraTreeRegressor = ExtraTreeRegressor(random_state=1)
param_ET = {
'max_depth':[5],
'max_features':['auto'],
'criterion' : ["friedman_mse"],
'splitter' : [ 'random']
}
gridsearch(model_ExtraTreeRegressor,param_ET,'Extra Tree')

Extra Tree
[7.45244530e-02 2.84858544e-02 2.61961909e-01 1.21168853e-05
 5.45183010e-02 8.73547704e-02 1.47081928e-02 1.63080715e-01
 0.00000000e+00 3.64540991e-02 1.95446425e-01 4.15693348e-02
 0.00000000e+00 2.43049636e-02 1.75788644e-02]
Best Regressor: {'criterion': 'friedman_mse', 'max_depth': 5, 'max_features': 'auto', 'splitter': 'random'} Best Score: 0.2893856540587009
(0.4163863821798082, 0.25913864854160135, 0.5090566260659037, 0.7355056692718165) TEST R2 0.7355056692718165 TEST CORR 0.86379
(0.31989877496578306, 0.22638118843944582, 0.4757953220024823, 0.7355876802782408) TRAIN R2 0.7355876802782408 TRAIN CORR 0.88932
finished


In [35]:
shap_plot(model_ExtraTreeRegressor,param_ET,'Extra Tree')

Extra Tree
[8.04966021e-02 4.08767661e-02 3.37782110e-01 2.93937830e-04
 6.70870476e-02 1.52016156e-01 5.73624409e-02 1.55788624e-01
 0.00000000e+00 3.63800951e-02 3.57431514e-01 4.26164385e-02
 0.00000000e+00 2.01946803e-02 1.33642906e-02]


In [36]:
###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeRegressor = tree.DecisionTreeRegressor(random_state=1)
param_DT = {
'max_depth':[5],
'max_features':['auto'],
'criterion' : ["friedman_mse"],
'splitter' : [ 'random']
}
gridsearch(model_DecisionTreeRegressor,param_DT,'Decision Tree')

Decision Tree
[7.45244530e-02 2.84858544e-02 2.61961909e-01 1.21168853e-05
 5.45183010e-02 8.73547704e-02 1.47081928e-02 1.63080715e-01
 0.00000000e+00 3.64540991e-02 1.95446425e-01 4.15693348e-02
 0.00000000e+00 2.43049636e-02 1.75788644e-02]
Best Regressor: {'criterion': 'friedman_mse', 'max_depth': 5, 'max_features': 'auto', 'splitter': 'random'} Best Score: 0.2893856540587009
(0.4163863821798082, 0.25913864854160135, 0.5090566260659037, 0.7355056692718165) TEST R2 0.7355056692718165 TEST CORR 0.86379
(0.31989877496578306, 0.22638118843944582, 0.4757953220024823, 0.7355876802782408) TRAIN R2 0.7355876802782408 TRAIN CORR 0.88932
finished


In [37]:
shap_plot(model_DecisionTreeRegressor,param_DT,'Decision Tree')

Decision Tree
[8.04966021e-02 4.08767661e-02 3.37782110e-01 2.93937830e-04
 6.70870476e-02 1.52016156e-01 5.73624409e-02 1.55788624e-01
 0.00000000e+00 3.63800951e-02 3.57431514e-01 4.26164385e-02
 0.00000000e+00 2.01946803e-02 1.33642906e-02]


In [38]:
###########AdaBoost gridsearch CV for best hyperparameter##########
model_AdaBoostRegressor = ensemble.AdaBoostRegressor(random_state=1)
param_Ada = {
'n_estimators':[50],
'learning_rate':[0.04],
'loss':['linear']
}
gridsearch(model_AdaBoostRegressor,param_Ada,'AdaBoost')

AdaBoost
[0.00230925 0.04263253 0.38883962 0.01327721 0.01132864 0.26019213
 0.03299807 0.03781186 0.03837857 0.03204152 0.01950473 0.01046481
 0.00102759 0.01620238 0.09299107]
Best Regressor: {'learning_rate': 0.04, 'loss': 'linear', 'n_estimators': 50} Best Score: 0.40840082792053395
(0.2797673552963023, 0.11384986774597815, 0.3374164603957225, 0.8374407177996199) TEST R2 0.8374407177996199 TEST CORR 0.93827
(0.3705947324836792, 0.21219291455381167, 0.46064402151098377, 0.6808573566981495) TRAIN R2 0.6808573566981495 TRAIN CORR 0.90484
finished


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [39]:
shap_plot(model_AdaBoostRegressor,param_Ada,'AdaBoost')

AdaBoost


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


[0.00307837 0.03484201 0.37260721 0.01004378 0.0167947  0.21609052
 0.04556119 0.04472817 0.05774204 0.01817255 0.01564384 0.01060723
 0.00083112 0.01200027 0.08603272]
